# 💻 Tutorial de FEniCS (para el problema de elasticidad no lineal)

## Tutorial original por David Ortiz
## Versión actualizada por Pablo Zurita Soler (pzurita@uc.cl)

FEniCS es un proyecto colaborativo de _software_ que provee una interfaz simbólica intuitiva a un sistema eficiente de resolución de ecuaciones diferenciales parciales usando el método de elementos finitos. Para más detalles, ver [la página principal](https://fenicsproject.org/), y, en especial

### [el tutorial en HTML](https://fenicsproject.org/pub/tutorial/html/ftut1.html), con códigos de ejemplo para otras ecuaciones y explicaciones útiles

### [la documentación de la última versión](https://fenics.readthedocs.io/en/latest/), ojo con que sea la versión que estén usando

Para usar FEniCS corriendo en su computador, debe ser instalado sobre una distribución de Python cualquiera en Linux, o sobre Anaconda en MacOS. Para uso en Windows, se debe instalar WSL para tener una consola de Linux nativa. Todos los detalles se encuentran en la página principal de FEniCS.

Alternativamente, **pueden correr FEniCS en un ambiente de Google Colab**. Para esto, deben correr el siguiente código (al comienzo de su archivo `.ipynb`):

In [ ]:
# Add FEniCS PPA to Ubuntu and install (Python3.6)
!echo deb http://ppa.launchpad.net/fenics-packages/fenics/ubuntu bionic main >> /etc/apt/sources.list
!apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 2C5275D7EF63D9DE2D28D3702940F5212B746472
!apt-get -qq update

!apt-get -y install fenics > /dev/null

# Reinstall dolfin for Python3.7
!rm -r /usr/lib/petsc/lib/python3/dist-packages/dolfin
!git clone https://bitbucket.org/fenics-project/dolfin
!cd dolfin/python && python3 setup.py install

# Reinstall mpi4py for Python3.7
!rm -r /usr/lib/python3/dist-packages/mpi4py*
!pip3 install mpi4py --upgrade

y, una vez terminado de instalar, deben ejecutar la instrucción de _restart runtime_. Si se siguieron esos pasos, el siguiente comando debería correr sin problemas. Este importa la librería fundamental de FEniCS, llamada `dolfin`.

In [ ]:
from dolfin import *

Ahora vamos al tutorial del problema de hiperelasticidad.

# Hiperelasticidad

Sea $B_0 \subset \mathbb{R}^3$ un cuerpo con densidad de masa $R$ que asumiremos constante. Adicionalmente, definimos

- $\varphi:B_0\to\mathbb{R}^3$ como el mapeo de deformación (desconocido).
- $\bar{\varphi}:\partial B_1\to\mathbb{R}^3$ como el mapeo de deformación preescrito en frontera Dirichlet.
- $\bar{T}: \partial B_2\to\mathbb{R}^3$ como las tracciones prescritas en frontera Neumann.
- $\boldsymbol{P}\in CT(2)$ como el primer tensor de Piola-Kirchoff. 
- $\boldsymbol{B}: B_0 \to \mathbb{R}^3$ como la densidad material de fuerzas de cuerpo.
- $u := \boldsymbol{\varphi} - \mathrm{id}$ como el mapeo de desplazamientos.

Recordemos que para un material hiperelástico nuestra ley constitutiva será $\boldsymbol{P}=\frac{\partial W}{\partial \boldsymbol{F}}$, donde $W(\boldsymbol{F})$ es la densidad de energía de deformación por unidad de volumen. Del balance de la formulación material y del balance de momentum lineal, el problema fuerte se define como sigue: encontrar $\boldsymbol{\varphi}\in H^2(B_0,\mathbb{R}^3)$ tal que

\begin{align}
    & \nabla_0 \cdot \boldsymbol{P} + R \boldsymbol{B} = 0 \quad && \forall \boldsymbol{X}\in B_0 \\
    & \boldsymbol{P} = \frac{\partial W}{\partial F}, \quad && \forall \boldsymbol{X}\in B_0 \\
    &\boldsymbol{\varphi}(\boldsymbol{X}) = \bar{\boldsymbol{\varphi}}(\boldsymbol{X}),\quad && \forall \boldsymbol{X}\in\partial B_1 \\
    & \boldsymbol{PN}=\bar{\boldsymbol{T}},\quad && \forall\mathbf{X}\in\partial B_2 \\
\end{align}

donde $\partial B=\partial B_1\cup\partial B_2$ y $\partial B_1\cap\partial B_2=\emptyset$. $\partial B_1$ se denomina la frontera Dirichlet, porque en ella se preescribe el valor directo de la función que estamos estimando ($\boldsymbol{\varphi}$), y esto es una condición de borde de Dirichlet. $\partial B_2$ se denomina la frontera Neumann, pues en ella se preescrible el valor de la derivada normal de la función que estamos estimando ($\boldsymbol{F}$ aparece a través de $\boldsymbol{PN}$), y esto es una condición de borde de Neumann.

Para solucionar este problema, utilizaremos el principio de trabajos virtuales descrito en la clase 12, pero le haremos una pequeña modificación. 

Un mapeo de deformación $\boldsymbol{\varphi}: B_0 \to\mathbb{R}^3$ resuelve el problema de elasticidad no lineal si y solo si para este se cumple el principio de los trabajos virtuales

$$
\int_{B_0}\boldsymbol{P}(\nabla_0\boldsymbol{\varphi}) : \nabla_0\boldsymbol{\eta} \,  \mathrm{d}\boldsymbol{X} - \int_{B_0} R \boldsymbol{B} \cdot \boldsymbol{\eta} \, \mathrm{d}\boldsymbol{X} - \int_{\partial B_2} \bar{\boldsymbol{T}} \cdot \boldsymbol{\eta} \, \mathrm{d} S = 0 \qquad \forall\boldsymbol{\eta} \in \mathcal{V} \text{.}
$$

Se puede demostrar (se ve en cursos como Elementos Finitos o Mecánica del Continuo) que la solución $\boldsymbol{\varphi}$ del PTV es de hecho la solución de un problema de optimización, que corresponde a minimizar un funcional de energía. Para esto solo definiremos el espacio de funciones de prueba $\mathcal{S} := \{\boldsymbol{\varphi} \in H^1(B_0,\mathbb{R}^3) \mid  \left.\boldsymbol{\varphi}\right\rvert_{\partial B_1} = \bar{\boldsymbol{\varphi}}\}$ y la energía potencial de un cuerpo

$$
\Pi[\boldsymbol{\varphi}] := \int_{B_0} W(\boldsymbol{F}) \, \mathrm{d}\boldsymbol{X} - \int_{B_0} R \boldsymbol{B}\cdot \boldsymbol{\varphi} \, \mathrm{d}\boldsymbol{X} - \int_{\partial B_2} \bar{\boldsymbol{T}} \cdot \boldsymbol{\varphi} \, \mathrm{d}S
$$

por lo que nuestro objetivo será encontrar un mapeo de deformación $\boldsymbol{\varphi}\in\mathcal{S}$ que minimice el potencial de energía $\Pi[\boldsymbol{\varphi}]$:

$$
\min_{\boldsymbol{\varphi}\in\mathcal{S}}\Pi[\boldsymbol{\varphi}]
$$

Adoptaremos una densidad de energía de deformación para material neo-hookeano

$$
W(F)=\frac{\mu}{2}(tr(C)-3)-\mu \ln(J)+\frac{\lambda}{2}\ln(J)^2
$$

con parámetros de Lamé

$$
\lambda= \frac{E\nu}{(1+\nu)(1-2\nu)}, \qquad \mu=\frac{E}{2(1+\nu)}
$$

donde $E$ es el modulo de Young y $\nu$ el coeficiente de Poisson.

### Solución numérica con elementos finitos

Solucionaremos el problema de hiperelasticidad en un cuboide rectangular de dimensiones $3\times 1\times 1$ (dominio $B_0\in\mathbb{R}^3$), donde la densidad material de fuerzas de cuerpo será una constante negativa sobre el eje $z$. El objeto estará empotrado por lo que tendrá desplazamientos preescritos nulos en la frontera $x=0$ y una torsión en la frontera $x=3$. Tendrá tracciones preescritas en la frontera superior $z=1$.

$$\mathbf{B}= (0,0,0)$$

$$u_0 = (0,0,0)$$

$$u_1 = (0,\quad0.5+(y-0.5)\cos(\pi/3))-(z-0.5)\sin(\pi/3)-y)/2,\quad0.5+(y-0.5)\sin(\pi/3))+(z-0.5)\cos(\pi/3)-y)/2)$$

$$\bar{T}= (0,0,20 \sin(\pi x)),\qquad E=1e3,\qquad \nu=0.4$$

Comenzamos importando `dolfin`, la librería principal. Podemos opcionalmente cambiar parámetros de _back-end_ para mejorar la optimización del problema.

In [1]:
from dolfin import *

# Opciones de optimización para el compilador
q_degree = 3
parameters["form_compiler"]["quadrature_degree"] = q_degree
parameters["form_compiler"]["cpp_optimize"] = True
parameters["form_compiler"]["representation"] = "uflacs"

Generamos nuestro dominio.

In [2]:
# BoxMesh: Puntos de esquinas opuestas y numero de elementos en cada direccion
# 21 elementos en x, 7 en y y 7 en z

x_length = 3.0
mesh = BoxMesh(
    Point(0, 0, 0), Point(x_length, 1, 1), 21, 7, 7
)

Ahora debemos definir el espacio de funciones de prueba ($\mathcal{S}$). Es de resaltar que en el caso de hiperelasticidad la dimensión del problema cambia, por lo que pasamos de un espacio de funciones escalares a un espacio vectorial.

In [3]:
# Espacio de funciones a R^3 (Vector), sobre la malla (mesh),
# afín a tramos (polinomios de Langrange de grado 1)

S = VectorFunctionSpace(mesh, "Lagrange", 1)

Definimos las expresiones de borde del problema.

In [4]:
# Funciones de las fronteras y fuente
ubar0 = Expression(("0.0","0.0","0.0"), degree=1)

ubar1 = Expression(
    ("0.0", 
    "scale*(y0 + (x[1] - y0)*cos(theta) - (x[2] - z0)*sin(theta) - x[1])",
    "scale*(z0 + (x[1] - y0)*sin(theta) + (x[2] - z0)*cos(theta) - x[2])"),
    scale = 0.5, y0 = 0.5, z0 = 0.5, theta = pi/2, degree=1
)

Tbar = Expression(("0.0","0.0","20*sin(pi*x[0])"), degree=1)
RB = Constant((0.0, 0.0, 0.0))

Creamos las tres fronteras. Las fronteras de Dirichlet son agrupadas en una lista.

In [5]:
# Clases de las fronteras: se declaran todas las fronteras
# y se les ponen nombres segun su tipo. Noten que son clases que heredan de
# SubDomain, una clase madre que viene de dolfin.
# on_boundary es el espacio sobre toda la frontera, al ponerlo en conjunción
# con near(), están marcando las caras del borde que cumplan con la condición
# de estar cerca de lo que dicen.

class BoundaryDir0(SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary and near(x[0], 0)
    
class BoundaryDir1(SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary and near(x[0], x_length)
    
class BoundaryNeu(SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary and near(x[2], 1)

In [6]:
# Este comando asigna la etiqueta 0 a todas las fronteras
boundaries = MeshFunction("size_t", mesh, mesh.topology().dim()-2)
boundaries.set_all(0)

# Ahora asignamos las condiciones de borde en las respectivas fronteras

Dir_Boundary0 = BoundaryDir0()
Dir0 = DirichletBC(S, ubar0, Dir_Boundary0)  # Asigna ubar0 a Dir_Boundary0

Dir_Boundary1 = BoundaryDir1()
Dir1 = DirichletBC(S, ubar1, Dir_Boundary1)
BC = [Dir0, Dir1]

# La condición de Neumann no se asigna: aparece en la formulación integral

Neu_Boundary = BoundaryNeu()
Neu_Boundary.mark(boundaries, 0)

Definimos nuestras expresiones cinemáticas.

In [7]:
# Cinematica
u = Function(S)  # Instanciamos la función
d = len(u)  # Dimension del dominio
I = Identity(d)  # Tensor identidad
F = I + grad(u)  # Tensor gradiente de deformación
C = F.T*F  # Tensor derecho de Cauchy-Green 

Ic = tr(C)  # Primer invariante
J = det(F)  # Jacobiano

#Parámetros de Lame
E = 1e3
nu = 0.4  # ¡No es incompresible! La incompresibilidad trae problemas

mu = Constant(E/(2*(1 + nu)))  # Constant es para definir campos constantes
lmbda = Constant(E*nu/((1 + nu)*(1 - 2*nu)))

# Densidad de energia de deformación (compresible neo-Hookeano)
W = (mu/2)*(Ic - 3) - mu*ln(J) + (lmbda/2)*(ln(J))**2

Definimos nuestro potencial de energía.

In [8]:
# Diferencial de superficie. ds ya está definido, pero lo redefinimos para
# poder integrar sobre la partición de nuestro borde
ds = Measure('ds', domain=mesh, subdomain_data=boundaries)

# Energía potencial, con la integral sobre el borde
Pi = W*dx - dot(RB, u)*dx - dot(Tbar, u)*ds(0)

Aquí es donde se hace más notable la diferencia del problema de Poisson (el problema más fácil de resolver, que pueden ver en el tutorial de FEniCS) con el de hiperelasticidad. En el problema de Poisson resolvemos el sistema de ecuaciones lineales que quedan luego de discretizar nuestra formulación variacional. En el caso de hiperelasticidad, lo que queremos es encontrar el campo de desplazamientos que minimice el potencial $\Pi$ (un problema de optimización).

In [9]:
# Definimos nuestro problema variacional
du = TrialFunction(S)
v = TestFunction(S)  # Los \nu

# Primera variación (derivada direccional) de Pi
dPi = derivative(Pi, u, v)

# Jacobiano de dPi
J = derivative(dPi, u, du)

# Resolver el problema: optimización es "derivada = 0"
solve(dPi == 0, u, BC, J=J)


Luego de correr la línea anterior, `u` es ahora un objeto que contiene la función aproximada de $u$ sobre la malla. Por último, podremos almacenar nuestra solución en formato `.pvd`/`.vtu`/`.vtk`, que nos permitirá visualizar la información en ParaView. Para esto la proyectamos nuevamente a un espacio que cumpla con $H^1(\Omega,\mathbb{R})$ (la guardamos como una función "bonita" con "buenas" derivadas).

In [10]:
disp = project(u, VectorFunctionSpace(mesh, 'Lagrange', 1))  # Desplazamiento
disp.rename('u_field', 'u_field')

file = File("Hiper_disp.pvd")  # Nombre del archivo
file << disp  # Guardar la información

Con el campo de desplazamientos podemos obtener el primer tensor de Piola-Kirchhoff, el tensor de tensiones de Cauchy y las tensiones de von Mises (entre otras cosas). Recuerden que el primer tensor esta en configuración material, mientras que el segundo esta en configuración deformada. Para esto necesitamos definimos nuevamente nuestra cinemática, pero con el cambio ya conocido.

In [11]:
# Cinematica
uu = project(u, VectorFunctionSpace(mesh, 'DG', 1))  # D de discontinuo
F = I + grad(uu)  # Tensor gradiente de deformación
F = variable(F)

J = project(det(F), FunctionSpace(mesh, 'DG', 1))  # Ahora J es un campo

# Densidad de energia de deformacion (compresible neo-Hookeano)
W = (mu/2)*(tr(F.T*F)-3) - mu*ln(det(F)) + (lmbda/2)*(ln(det(F)))**2

# Definición de tensores
P = diff(W, F)  # Primer tensor PK, ¡computado de manera simbólica!
sigma = J**-1*P*F.T

# Tensiones de von Mises
s = sigma - (1. / 3.)*tr(sigma)*I  # Componente deviatórica
von_mises = sqrt(3. / 2.*inner(s, s))  # Tensión de von Mises como campo

Finalmente, almacenamos nuestras soluciones.

In [12]:
# Primer tensor de Piola-kirchhoff
PK = project(P, TensorFunctionSpace(mesh, 'DG', 0))  # Constante a trozos
PK.rename('stress_P', 'stress_P')
pvd = File('Hiper_PK.pvd')
pvd << PK

# Tensor de tensiones
sigma = project(sigma, TensorFunctionSpace(mesh, 'DG', 0))
sigma.rename('stress_sigma', 'stress_sigma')
pvd = File('Hiper_sigma.pvd')
pvd << sigma

# Tensiones de von Mises
von_mises = project(von_mises, FunctionSpace(mesh, 'DG', 0))
von_mises.rename('von_mises', 'von_mises')
pvd = File('Hiper_vm.pvd')
pvd << von_mises